Thermal analysis {#thermal_example}
================

PyDPF Composites can also be used to post-process thermal analyses. In
this case, the simulation is a two-step analysis where the results of a
thermal analysis are an input of the structural analysis. So, the RST
contains temperature and structural results. The example mimics a PCB
which was modeled with Ansys Composites PrePost (ACP). where the solid
model feature of ACP is used to generate the volume mesh.

In detail, the example shows how to extract the temperatures for a
specific ply, and a specific material.

::: {.note}
::: {.title}
Note
:::

When using a Workbench project, use the
`.get_composite_files_from_workbench_result_folder`{.interpreted-text
role="func"} method to obtain the input files.
:::


Set up analysis
===============

Setting up the analysis consists of loading the required modules,
connecting to the DPF server, and retrieving the example files.


In [ ]:
import ansys.dpf.core as dpf
import numpy as np

from ansys.dpf.composites.composite_model import CompositeModel
from ansys.dpf.composites.constants import TEMPERATURE_COMPONENT
from ansys.dpf.composites.example_helper import get_continuous_fiber_example_files
from ansys.dpf.composites.layup_info import get_all_analysis_ply_names
from ansys.dpf.composites.ply_wise_data import SpotReductionStrategy, get_ply_wise_data
from ansys.dpf.composites.select_indices import get_selected_indices_by_dpf_material_ids
from ansys.dpf.composites.server_helpers import connect_to_or_start_server

server = connect_to_or_start_server()
composite_files = get_continuous_fiber_example_files(server, "thermal_solid")

Initialize the model
====================

The composite model is initialized with the composite files and the
server. It provides access to the mesh, results, lay-up and materials


In [ ]:
composite_model = CompositeModel(composite_files, server)

Get Results - Temperatures
==========================

The temperatures are stored under structural\_temperature


In [ ]:
temp_op = composite_model.core_model.results.structural_temperature()
temperatures_fc = temp_op.outputs.fields_container()

Ply-wise results
================

Ply-wise results can be easily extracted using the function
`.get_ply_wise_data`{.interpreted-text role="func"} and by passing the
ply name.


In [ ]:
all_ply_names = get_all_analysis_ply_names(composite_model.get_mesh())
print(all_ply_names)

nodal_values = get_ply_wise_data(
    field=temperatures_fc,
    ply_name="P1L1__ModelingPly.8",
    mesh=composite_model.get_mesh(),
    component=TEMPERATURE_COMPONENT,
    spot_reduction_strategy=SpotReductionStrategy.MAX,
    requested_location=dpf.locations.nodal,
)

composite_model.get_mesh().plot(nodal_values)

Material-wise results
=====================

It is also possible to filter the results by material. In this example
the element-wise maximum temperature is extracted for the material
[Honeycomb Aluminum Alloy]{.title-ref}.


In [ ]:
print(composite_model.material_names)
material_id = composite_model.material_names["Honeycomb Aluminum Alloy"]

# get the last result field
temperatures_field = temperatures_fc[-1]

material_result_field = dpf.field.Field(location=dpf.locations.elemental, nature=dpf.natures.scalar)
# performance optimization: use a local field instead of a field which is pushed to the server
with material_result_field.as_local_field() as local_result_field:
    element_ids = temperatures_field.scoping.ids

    for element_id in element_ids:
        element_info = composite_model.get_element_info(element_id)
        assert element_info is not None
        if material_id in element_info.dpf_material_ids:
            temp_data = temperatures_field.get_entity_data_by_id(element_id)
            selected_indices = get_selected_indices_by_dpf_material_ids(element_info, [material_id])

            value = np.max(temp_data[selected_indices])
            local_result_field.append([value], element_id)

composite_model.get_mesh().plot(material_result_field)